In [121]:
import pandas as pd
from pandas import json_normalize
from sqlalchemy import create_engine
import requests
import json
import time


### Extract CSVs into DataFrames

In [22]:
hospital_data = "Resources/Medicare_Inpatient_Hospital_by_Provider_and_Service_2019.csv"

In [27]:
hospital_df = pd.read_csv(hospital_data)
hospital_df.head()

,Rndrng_Prvdr_CCN,Rndrng_Prvdr_Org_Name,Rndrng_Prvdr_St,Rndrng_Prvdr_City,Rndrng_Prvdr_State_Abrvtn,Rndrng_Prvdr_State_FIPS,Rndrng_Prvdr_Zip5,Rndrng_Prvdr_RUCA,Rndrng_Prvdr_RUCA_Desc,DRG_Cd,DRG_Desc,Tot_Dschrgs,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",14,"$326,514.86","$62,788.07","$61,050.29"
1,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,23,CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE COM...,55,"$140,874.53","$29,766.84","$27,205.18"
2,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,24,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,20,"$109,788.10","$22,780.30","$20,067.75"
3,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,25,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,23,"$124,579.26","$24,107.30","$22,764.74"
4,10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,27,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,16,"$75,029.06","$18,216.38","$10,206.88"


In [28]:
# Rename the column headers
hospital_df = hospital_df.rename(columns={"Rndrng_Prvdr_CCN": "provider_CCN",
                                                          "Rndrng_Prvdr_Org_Name": "org_name",
                                                          "Rndrng_Prvdr_St": "street_address", 
                                         "Rndrng_Prvdr_City": "city", 
                                         "Rndrng_Prvdr_State_Abrvtn": "state_abv", 
                                         "Rndrng_Prvdr_State_FIPS": "state_FIPS",
                                        "Rndrng_Prvdr_Zip5": "zip_code", 
                                         "Rndrng_Prvdr_RUCA" : "RUCA", 
                                         "Rndrng_Prvdr_RUCA_Desc" : "RUCA_desc", 
                                         "DRG_Cd" : "DRG_code", 
                                         "DRG_Desc" : "DRG_desc", 
                                         "Tot_Dschrgs" : "total_discharges", 
                                         "Avg_Submtd_Cvrd_Chrg" : "avg_submitted_covered_charges", 
                                         "Avg_Tot_Pymt_Amt" : "avg_total_payment_amnt", 
                                         "Avg_Mdcr_Pymt_Amt": "avg_medicare_payment_amnt"})


In [29]:
#set index to provider CCN and drop the column from the df (warning: only runs once in the kernel, don't re-run without reloading original DF)
hospital_df = hospital_df.set_index(hospital_df["provider_CCN"])
hospital_df = hospital_df.drop(columns = ["provider_CCN"])

#display dataframe 
hospital_df.head()

,org_name,street_address,city,state_abv,state_FIPS,zip_code,RUCA,RUCA_desc,DRG_code,DRG_desc,total_discharges,avg_submitted_covered_charges,avg_total_payment_amnt,avg_medicare_payment_amnt
provider_CCN,,,,,,,,,,,,,,
10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,3,"ECMO OR TRACH W MV >96 HRS OR PDX EXC FACE, MO...",14,"$326,514.86","$62,788.07","$61,050.29"
10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,23,CRANIOTOMY W MAJOR DEVICE IMPLANT OR ACUTE COM...,55,"$140,874.53","$29,766.84","$27,205.18"
10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,24,CRANIO W MAJOR DEV IMPL/ACUTE COMPLEX CNS PDX ...,20,"$109,788.10","$22,780.30","$20,067.75"
10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,25,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,23,"$124,579.26","$24,107.30","$22,764.74"
10001,Southeast Alabama Medical Center,1108 Ross Clark Circle,Dothan,AL,1,36301,1.0,Metropolitan area core: primary flow within an...,27,CRANIOTOMY & ENDOVASCULAR INTRACRANIAL PROCEDU...,16,"$75,029.06","$18,216.38","$10,206.88"


In [175]:
#get data from centers for medicare and medicaid services API and save to response variable 
#base_url = "https://data.cms.gov/data-api/v1/dataset/5fccd951-9538-48a7-9075-6f02b9867868/data"
base_url = "https://data.cms.gov/data-api/v1/dataset/5fccd951-9538-48a7-9075-6f02b9867868/data?column=Rndrng_NPI&column=Rndrng_Prvdr_Last_Org_Name&column=Rndrng_Prvdr_First_Name&column=Rndrng_Prvdr_St1&column=Rndrng_Prvdr_Type&column=Rndrng_Prvdr_State_Abrvtn&offset=1000&size=10000&limit=10000"
response = requests.get(base_url).json()

#verify that the call actually pulled the data 
print(len(response))

5000


In [108]:
#initialize empty lists to store the data from the API pull to eventually put into dataframe 
Rndrng_NPI = []
Rndrng_Prvdr_Last_Org_Name=[]
Rndrng_Prvdr_First_Name=[]
Rndrng_Prvdr_St1=[]
Rndrng_Prvdr_Type=[]
Rndrng_Prvdr_State_Abrvtn = []

In [109]:
#loop through JSON response and add data to the appropriate lists 

for x in range(len(response)): 
    try: 
        Rndrng_NPI.append(response[x]["Rndrng_NPI"])
        Rndrng_Prvdr_Last_Org_Name.append(response[x]["Rndrng_Prvdr_Last_Org_Name"])
        Rndrng_Prvdr_First_Name.append(response[x]["Rndrng_Prvdr_First_Name"])
        Rndrng_Prvdr_St1.append(response[x]["Rndrng_Prvdr_St1"])
        Rndrng_Prvdr_Type.append(response[x]["Rndrng_Prvdr_Type"])
        Rndrng_Prvdr_State_Abrvtn.append(response[x]["Rndrng_Prvdr_State_Abrvtn"])
    except: 
        print(f"There is an issue with the data row {x}. Skipping.")

In [110]:
#verify that the arrays are the same length in order to create dataframe 
print(len(Rndrng_NPI), len(Rndrng_Prvdr_Last_Org_Name), len(Rndrng_Prvdr_First_Name), len(Rndrng_Prvdr_St1), len(Rndrng_Prvdr_Type), len(Rndrng_Prvdr_State_Abrvtn))

1000 1000 1000 1000 1000 1000


In [115]:
#create dataframe based on the lists 
practitioner_df = pd.DataFrame({"NPI": Rndrng_NPI, "last_name" : Rndrng_Prvdr_Last_Org_Name,
                                "first_name": Rndrng_Prvdr_First_Name, "street_address":Rndrng_Prvdr_St1, 
                               "provider_type" : Rndrng_Prvdr_Type, "state_abv" : Rndrng_Prvdr_State_Abrvtn})
practitioner_df

,NPI,last_name,first_name,street_address,provider_type,state_abv
0,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
1,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
2,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
3,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
4,1003000126,Enkeshafi,Ardalan,900 Seton Dr,Internal Medicine,MD
...,...,...,...,...,...,...
995,1003007170,Danciu,Sorin,3118 N Ashland Ave,Cardiology,IL
996,1003007204,Yamani,Hussein,100 Medical Center Blvd,Cardiology,TX
997,1003007204,Yamani,Hussein,100 Medical Center Blvd,Cardiology,TX
998,1003007204,Yamani,Hussein,100 Medical Center Blvd,Cardiology,TX


### Transform premise DataFrame

In [4]:
# Create a filtered dataframe from specific columns
premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
premise_transformed= premise_df[premise_cols].copy()

# Rename the column headers
premise_transformed = premise_transformed.rename(columns={"License Serial Number": "id",
                                                          "Premises Name": "premise_name",
                                                          "County ID Code": "county_id"})

# Clean the data by dropping duplicates and setting the index
premise_transformed.drop_duplicates("id", inplace=True)
premise_transformed.set_index("id", inplace=True)

premise_transformed.head()

,premise_name,county_id
id,,
1311660,CANA ARRIBA GROCERY NO 2 INC,0
1310023,JOHANA GROCERY & DELI CORP,0
1310024,TKO BEVERAGES LLC,1
1311663,181 LEXINGTON AVENUE BBQ LLC,2
1310029,AZIZ DELI & GRILL CORP,1


### Transform county DataFrame

In [5]:
county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
county_transformed = county_df[county_cols].copy()

# Rename the column headers
county_transformed = county_transformed.rename(columns={"ID": "id",
                                                         "County Name (Licensee)": "county_name",
                                                         "License Count": "license_count",
                                                         "County ID Code": "county_id"})

# Set index
county_transformed.set_index("id", inplace=True)

county_transformed.head()

,county_name,county_id,license_count
id,,,
0,ALBANY,5,77
1,ALLEGANY,59,4
2,BRONX,0,104
3,BROOME,35,14
4,CATTARAUGUS,41,9


### Create database connection

In [6]:
connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
# Confirm tables
engine.table_names()

['county', 'premise']

### Load DataFrames into database

In [8]:
premise_transformed.to_sql(name='premise', con=engine, if_exists='append', index=True)

In [9]:
county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)